# Taller 1 Conceptos básicos

## 1. Introducción a NLP

### 1.1 Ejercicio: Investigar 3 aplicaciones actuales de NLP

### 1.2 Ejercicio: Usar la biblioteca nltk o spaCy en Python para tokenizar un texto corto.

#### 1.2.1 Ejercicio con Nltk

##### 1.2.1.1 Intalación de NLTK

In [1]:
!pip install nltk

##### 1.2.1.2 Importar las librerías

In [2]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

C:\Users\guill\anaconda3\lib\site-packages\scipy\__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 2.0.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('wordnet')

In [ ]:
nltk.download("omw-1.4")

##### 1.2.1.3 Escribir un texto

In [ ]:
nltk_text = "I am learning about tokenization!"

##### 1.2.1.3 Tokenizar por oraciones

In [ ]:
sentences = sent_tokenize(nltk_text)
print(f"Sentance list: {sentences}.")

##### 1.2.1.4 Tokenizar por palabras

In [ ]:
words = word_tokenize(nltk_text)
print(f"Word list: {words}.")

##### 1.2.1.5 Tokenizar por subpalabras

In [ ]:
stemmer = PorterStemmer()
stemmer_subwords = [stemmer.stem(word) for word in words]
print(f"Subword list: {stemmer_subwords}.")

In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatizer_subwords = [lemmatizer.lemmatize(word) for word in words]
print(f"Subword list: {lemmatizer_subwords}.")

##### 1.2.1.6 Tokenizar por caracteres

In [ ]:
characters = list(text)
print(f"Character List: {characters}")

#### 1.2.2 Ejercicio con spaCy

##### 1.2.2.1 Intalación de NLTK

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

##### 1.2.2.2 Importar las librerías

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

##### 1.2.2.3 Escribir un texto

In [ ]:
spacy_text = "I am learning how to do tokenization with spacy. It is awesome!"

In [ ]:
for sent in doc.sents:
    print(sent.text)

In [ ]:
words = []
for token in doc:
    words.append(token.text)
words

In [ ]:
characters = list(spacy_text)
characters

## 2. Embeddings y Representaciones Vectoriales

### Ejercicio: Cargar un modelo de word embeddings de Hugging Face (por ejemplo, glove o word2vec) y visualizar los embeddings de palabras similares

### Ejercicio: Comparar GloVe (estático) con BERT (dinámico) mediante un ejemplo práctico en Python.

## 3. Modelos Basados en Redes Neuronales

### Ejercicio: Implementar una RNN simple para predicción de secuencias en Python.

### Ejercicio: Realizar el fine-tuning de un modelo pre-entrenado de Hugging Face (BERT o RoBERTa) para completar una frase simple.

## 4. Modelos Pre-entrenados y Fine-Tuning

### Ejercicio: Usar BERT para predecir palabras enmascaradas en una oración

### Ejercicio: Usar un modelo de Hugging Face (distilBERT) para clasificar un conjunto de datos de sentimientos (usar el dataset de Rotten tomatoes).

## 5. Tareas Clásicas de NLP

### Ejercicio: Utilizar Hugging Face para hacer análisis de sentimientos en un conjunto de datos de reseñas (usar el dataset de IMDB).

### Ejercicio: Implementar un modelo NER utilizando spaCy o Hugging Face y aplicar etiquetas a un conjunto de texto.

## 6. Generación de Texto

### Ejercicio: Usar Hugging Face para generar texto a partir de un prompt con GPT-2.

### Ejercicio: Discutir sobre sesgos en los modelos de lenguaje generativo y su impacto en aplicaciones reales.

## 7. Evaluación de Modelos NLP

### Ejercicio: Usar métricas como accuracy, precision, y recall para evaluar un modelo de clasificación de texto.

### Ejercicio: Calcular la perplejidad de un modelo de lenguaje en Hugging Face para una tarea de generación de texto.